In [1]:
import taxon
import gui_widgets
from wikidataintegrator import wdi_core


In [34]:
import ipywidgets as widgets
from IPython.display import IFrame
tab1 = widgets.Output()
tab2 = widgets.Output()
tab3 = widgets.Output()
tab4 = widgets.Output()
tab5 = widgets.Output()
tab = widgets.Tab(children=[tab1,tab2, tab3, tab4, tab5])
tab

iNaturalistTab = IFrame(src='https://www.inaturalist.org/home', width=1000, height=600)
tab.set_title(0, 'iNaturalist') 
tab.set_title(1, 'GBIF')
tab.set_title(2, 'Wikidata')
tab.set_title(3, 'Commons')
tab.set_title(4, 'Wikipedia')

with tab1:
    def paste_commons(commons_file_name):
        return commons_file_name
    def get_data(taxon_id):
        global data
        data = taxon.external_data(inaturalist_id=taxon_id)
        html = "<img src='"+data.inaturalist_data[0]['default_photo']['medium_url']+"'><br>"+data.inaturalist_data[0]['default_photo']['attribution']
        output_widget = widgets.HTML(value=html)
        with tab2:
            html2 = "<table>"
            for key in data.gbif_data.keys():
                html2 += "<tr><td>{}</td><td>{}</td></tr>".format(key, data.gbif_data[key])
            html2 += "</table>"
            gbif_output = widgets.HTML(value=html2)
            display(gbif_output)
        with tab4:
            commons_query = """
                SELECT * WHERE {{?commons schema:about <{taxon}>  ; 
                schema:isPartOf <https://commons.wikimedia.org/> . 
                }}""".format(taxon = data.wikidata["main_rank"].loc[0]["taxon"])
            commons_query_result = wdi_core.WDItemEngine.execute_sparql_query(commons_query, as_dataframe=True)
            if len(commons_query_result) == 0:
                html4 = "<a href = 'https://commons.wikimedia.org/w/index.php?title=Category:"+data.inaturalist_data[0]["name"].replace(" ", "_")+"&action=edit'>create commons category</a><br>"
                html4 += "[[Category:"+data.inaturalist_data[0]["name"].replace(" ", "|")+"]]"
            else:
                html4 = "<a href = '"+data.wikidata["main_rank"]["commons"].loc[0]+"' target='_new'>"+data.wikidata["main_rank"]["commons"].loc[0]+"</a><br>"
            commons_output = widgets.HTML(value=html4)
            
            data.selected_commons=gui_widgets.interact_manual(paste_commons, commons_file_name="")
            display(commons_output)
        with tab5:
            print("https://en.wikipedia.org/wiki/"+data.inaturalist_data[0]["name"].replace(" ", "_"))
            print("=========================")
            print(data.create_wikipedia_stub(infobox_image=data.selected_commons.widget.result)) 
        return output_widget

    
    data = None
    taxon_window = gui_widgets.interact_manual(get_data, taxon_id="")
    
    
    



display(tab)

In [7]:
data.wikidata["main_rank"]["commons"].loc[0]

'https://commons.wikimedia.org/wiki/Category:Ypthima_sesara'

In [30]:
data.selected_commons.widget.result

'Ypthima sesara 31615442.jpg'